In [1]:
%pip install --upgrade --quiet  langchainhub   langchain-chroma bs4


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# 无聊天记录的链

In [5]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_community.embeddings.cloudflare_workersai import CloudflareWorkersAIEmbeddings
from supabase.client import Client, create_client

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0.7,
    streaming=True,
)
embeddings = CloudflareWorkersAIEmbeddings(
    account_id=os.getenv('CF_ACCOUNT_ID'),
    api_token=os.getenv('CF_API_TOKEN'),
    model_name="@cf/baai/bge-small-en-v1.5",
)

# supabase_url = os.environ.get("SUPABASE_URL")
# supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
# 
# supabase: Client = create_client(supabase_url, supabase_key)

In [6]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

persist_directory = '/Users/pangmengting/Documents/workspace/python-learning/langchain/history/chroma-data'
collection_name = 'history_index'
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings,
                                    collection_name=collection_name,
                                    persist_directory=persist_directory)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | qw_llm_openai
        | StrOutputParser()
)

2024-07-04 18:08:54,489:INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [7]:
rag_chain.invoke("What is Task Decomposition?")

2024-07-04 18:09:23,037:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


'Task Decomposition is a method used to break down complex tasks into smaller, simpler steps. This technique involves chaining of thoughts and generating multiple thoughts per step, creating a tree structure. It can be done manually through simple prompts or automatically using task-specific instructions or human inputs.'

In [ ]:
# 将问题置于背景中

# 首先，我们需要定义一个子链，它接受历史消息和最新的用户问题，如果它引用了历史信息中的任何信息，则重新定义问题。/

# 我们将使用一个包含名为“chat_history”的 MessagesPlaceholder 变量的提示符。这允许我们使用“chat_history”输入键向提示符传递消息列表，
# 这些消息将插入在系统消息之后，包含最新问题的人工消息之前。

# 请注意，我们在此步骤中使用了一个辅助函数create_history_aware_retriever，它管理 chat_history 为空的情况，
# 否则按顺序应用 prompt | llm | StrOutputParser() | retriever 。

# create_history_aware_retriever 构造了一个链，接受键 input 和 chat_history 作为输入，并具有与检索器相同的输出模式。

In [8]:
from langchain.chains.history_aware_retriever import create_history_aware_retriever
# from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# =====
# 中文: 给定一个聊天历史记录和最新的用户问题，该问题可能参考了聊天历史记录中的上下文，制定一个独立的问题，
# 这个问题无需聊天历史记录就能理解。不要回答这个问题，如果需要，只需重新表述它，否则就按原样返回。
# =====
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    qw_llm_openai, retriever, contextualize_q_prompt
)

In [ ]:
# 带聊天记录的链

# 现在我们可以建立完整的QA链。

# 在这里，我们使用create_stuff_documents_chain生成一个 question_answer_chain ，
# 输入键为 context 、 chat_history 和 input --它接受检索到的上下文以及会话历史和查询，以生成一个答案。

# 我们使用create_retrieval_chain构建最终的 rag_chain 。这个链按顺序应用 history_aware_retriever 和 question_answer_chain ，
# 为了方便起见，保留了中间输出，
# 如检索到的上下文。它有输入键 input 和 chat_history ，并在其输出中包括 input 、 chat_history 、 context 和 answer 。

In [9]:
from langchain.chains.retrieval import create_retrieval_chain
# from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# =====
# -  🔤 中文: 你是回答问题任务的助手。使用以下检索到的上下文片段来回答问题。如果你不知道答案，就说你不知道。最多使用三句话并保持答案简洁。
# =====
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(qw_llm_openai, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [10]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

2024-07-04 18:22:51,014:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


In [11]:
ai_msg_1

{'input': 'What is Task Decomposition?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?'),
  'Task Decomposition is a method used to break down complex tasks into smaller, more manageable ones. It involves breaking down a problem into multiple steps or subtasks and then analyzing the logic behind each step. This allows the agent to focus on specific aspects of the task while also considering the overall goal. It is commonly used in artificial intelligence and machine learning applications to help agents make sense of large amounts of data and perform tasks more efficiently.'],
 'context': [Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instr

In [12]:
second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

2024-07-04 18:29:50,023:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-04 18:29:52,439:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


There are several ways to do task decomposition:

  1. Using LLMs with simple prompts such as "Steps for XYZ" and "What are the subgoals for achieving XYZ."
  2. Using task-specific instructions like "Write a story outline" for writing a novel or "Create a list of items needed for cooking dinner" for meal preparation.
  3. Human inputs to guide the decomposition process.

Overall, task decomposition helps agents break down complex tasks into smaller, more manageable parts, allowing them to focus on specific aspects of the task while still considering the overall goal.


In [13]:
chat_history

[HumanMessage(content='What is Task Decomposition?'),
 'Task Decomposition is a method used to break down complex tasks into smaller, more manageable ones. It involves breaking down a problem into multiple steps or subtasks and then analyzing the logic behind each step. This allows the agent to focus on specific aspects of the task while also considering the overall goal. It is commonly used in artificial intelligence and machine learning applications to help agents make sense of large amounts of data and perform tasks more efficiently.']

In [14]:
chat_history.extend([HumanMessage(content=second_question), ai_msg_2["answer"]])

In [15]:
chat_history

[HumanMessage(content='What is Task Decomposition?'),
 'Task Decomposition is a method used to break down complex tasks into smaller, more manageable ones. It involves breaking down a problem into multiple steps or subtasks and then analyzing the logic behind each step. This allows the agent to focus on specific aspects of the task while also considering the overall goal. It is commonly used in artificial intelligence and machine learning applications to help agents make sense of large amounts of data and perform tasks more efficiently.',
 HumanMessage(content='What are common ways of doing it?'),
 'There are several ways to do task decomposition:\n\n  1. Using LLMs with simple prompts such as "Steps for XYZ" and "What are the subgoals for achieving XYZ."\n  2. Using task-specific instructions like "Write a story outline" for writing a novel or "Create a list of items needed for cooking dinner" for meal preparation.\n  3. Human inputs to guide the decomposition process.\n\nOverall, tas

In [16]:
second_question = "什么是rag?"
ai_msg_3 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_3["answer"])

2024-07-04 18:34:28,612:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-04 18:34:32,043:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


RAG是一种基于规则的生成模型，它通过使用规则来生成文本。它的名字是Random Access Grammar，其中Access代表了可以访问的资源（如单词表、词典等），Grammar则表示生成规则。

RAG模型通常用于自动问答系统中，因为它可以根据用户的问题生成答案。它可以将问题分解成一系列步骤，并在每个步骤中选择一个合适的输入或输出。这使得机器能够理解和回答复杂的问题，而不需要人类的帮助。


In [17]:
ai_msg_3

{'input': '什么是rag?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?'),
  'Task Decomposition is a method used to break down complex tasks into smaller, more manageable ones. It involves breaking down a problem into multiple steps or subtasks and then analyzing the logic behind each step. This allows the agent to focus on specific aspects of the task while also considering the overall goal. It is commonly used in artificial intelligence and machine learning applications to help agents make sense of large amounts of data and perform tasks more efficiently.',
  HumanMessage(content='What are common ways of doing it?'),
  'There are several ways to do task decomposition:\n\n  1. Using LLMs with simple prompts such as "Steps for XYZ" and "What are the subgoals for achieving XYZ."\n  2. Using task-specific instructions like "Write a story outline" for writing a novel or "Create a list of items needed for cooking dinner" for meal preparation.\n  3. Human inputs to guide t

In [19]:
second_question = "什么是rag?"
ai_msg_4 = rag_chain.invoke({"input": second_question, "chat_history": []})

print(ai_msg_4["answer"])

2024-07-04 18:36:02,432:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


RAG (Randomly Generated Assistant) 是一种人工智能技术，它结合了生成式模型和有监督学习。RAG 使用随机文本作为输入来训练一个基于上下文的机器翻译系统，并使用已有的语料库进行微调。

RAG 的核心思想是将生成器（Generator）与解释器（Inverter）组合在一起，使得在生成文本时可以同时考虑生成内容的质量和语言结构的正确性。这种设计方式允许 RAG 在处理自然语言任务时能够自动生成高质量的内容，而无需显式地指定每一步的逻辑或规则。

RAG 模型通常由两部分组成：生成器和解释器。生成器负责根据给定的初始文本生成后续的文本；解释器则用于评估生成文本的质量并提供反馈。这种设计模式使得 RAG 能够有效地利用现有的语言资源，如大量的训练数据、预训练模型等，从而快速提高其性能。


In [20]:
ai_msg_4

{'input': '什么是rag?',
 'chat_history': [],
 'context': [Document(page_content='Conversatin samples:\n[\n  {\n    "role": "system",', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  Document(page_content='Here are a sample conversation for task clarification sent to OpenAI ChatCompletion endpoint used by GPT-Engineer. The user inputs are wrapped in {{user input text}}.\n[\n  {\n    "role": "system",\n    "content": "You will read instructions and not carry them out, only seek to clarify them.\\nSpecifically you will first summarise a list of super short bullets of areas that need clarification.\\nThen you will pick one clarifying question, and wait for an answer from the user.\\n"\n  },\n  {\n    "role": "user",\n    "content": "We are writing {{a Super Mario game in python. MVC components split in separate files. Keyboard control.}}\\n"\n  },\n  {\n    "role": "assistant",', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  D

In [ ]:
 # 回返来源

# 通常在问答应用程序中，向用户显示用于生成答案的来源是很重要的。
# LangChain内置的 create_retrieval_chain 会将检索到的源文档传播到 "context" 键中的输出：

In [22]:
for document in ai_msg_4["context"]:
    print(document)
    print()

page_content='Conversatin samples:\n[\n  {\n    "role": "system",' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

page_content='Here are a sample conversation for task clarification sent to OpenAI ChatCompletion endpoint used by GPT-Engineer. The user inputs are wrapped in {{user input text}}.\n[\n  {\n    "role": "system",\n    "content": "You will read instructions and not carry them out, only seek to clarify them.\\nSpecifically you will first summarise a list of super short bullets of areas that need clarification.\\nThen you will pick one clarifying question, and wait for an answer from the user.\\n"\n  },\n  {\n    "role": "user",\n    "content": "We are writing {{a Super Mario game in python. MVC components split in separate files. Keyboard control.}}\\n"\n  },\n  {\n    "role": "assistant",' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

page_content='You should only respond in JSON format as described below\nResponse 

In [ ]:
# 在这里，我们已经讨论了如何添加应用程序逻辑来合并历史输出，但我们仍然手动更新聊天历史并将其插入到每个输入中。
# 在一个真实的Q&A应用程序中，我们需要一些方法来保存聊天记录，以及一些自动插入和更新聊天记录的方法。

# 为此，我们可以用途：

# BaseChatMessageHistory：存储聊天记录。
# RunnableWithMessageHistory：LCEL链和 BaseChatMessageHistory 的包装器，用于处理将聊天历史注入输入并在每次调用后更新它。



In [ ]:
# 实现第二个选项的一个简单示例，其中聊天历史记录存储在一个简单的dict中。

In [23]:
import bs4
# from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

persist_directory = '/Users/pangmengting/Documents/workspace/python-learning/langchain/history/chroma-data'
collection_name = 'history2_index'
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings,
                                    collection_name=collection_name,
                                    persist_directory=persist_directory)
retriever = vectorstore.as_retriever()

### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    qw_llm_openai, retriever, contextualize_q_prompt
)

### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(qw_llm_openai, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Statefully manage chat history ###
# 有状态地管理聊天记录
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# RunnableWithMessageHistory 允许我们将消息历史添加到某些类型的链中。它包装另一个Runnable并管理它的聊天消息历史记录。
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [24]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

2024-07-04 18:48:18,576:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


'Task Decomposition is a method used to break down complex tasks into smaller, more manageable ones. It involves breaking down a task into multiple steps or subtasks and then analyzing the logic behind each step. This allows the agent to focus on specific aspects of the task while also ensuring that it completes all necessary steps.'

In [25]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content='Task Decomposition is a method used to break down complex tasks into smaller, more manageable ones. It involves breaking down a task into multiple steps or subtasks and then analyzing the logic behind each step. This allows the agent to focus on specific aspects of the task while also ensuring that it completes all necessary steps.')])}

In [26]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

2024-07-04 18:48:48,893:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-04 18:48:52,750:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


'There are several common ways to perform task decomposition:\n\n  * Using LLMs with simple prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ."\n  * Using task-specific instructions such as "Write a story outline" for writing a novel.\n  * Using human inputs to guide the decomposition process.\n\nThese approaches help to make the task more manageable and allow the agent to focus on specific aspects of the task while still completing all necessary steps.'

In [27]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content='Task Decomposition is a method used to break down complex tasks into smaller, more manageable ones. It involves breaking down a task into multiple steps or subtasks and then analyzing the logic behind each step. This allows the agent to focus on specific aspects of the task while also ensuring that it completes all necessary steps.'), HumanMessage(content='What are common ways of doing it?'), AIMessage(content='There are several common ways to perform task decomposition:\n\n  * Using LLMs with simple prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ."\n  * Using task-specific instructions such as "Write a story outline" for writing a novel.\n  * Using human inputs to guide the decomposition process.\n\nThese approaches help to make the task more manageable and allow the agent to focus on specific aspects of the task while still completing all ne

In [ ]:
# 文档地址
# https://python.langchain.com/v0.1/docs/use_cases/question_answering/chat_history/